In [1]:
# in case there are any problems with importing because path is wrong
import sys
sys.path.append('/Users/daniel/Princeton Dropbox/Daniel Gurevich/Research/discrete_sr/code/SPIDER_discrete')

import numpy as np
from sparse_reg_bf import *
from sparse_reg import regress

In [2]:
w = 100
h = 150
v = np.zeros([w, 1])
k = 10
#v[0:10, 0] = [1, 1, 1, 1, 1, 1, 0.8, 0.6, 0.4, 0.2]
v[0:10, 0] = np.array(range(10, 0, -1))/10

def generate_random_Q(inv_snr):
    # inv_snr: (signal to noise ratio)^-1
    U = np.random.normal(size=[h, w])
    W = np.random.normal(size=[h, w])
    #U = np.random.uniform(0, 1, [w, h])
    
    #Sigma = (U.T @ U)/p + snr/(np.linalg.norm(v)**2)*(v @ v.T)
    
    true_matrix = W - (W @ v @ v.T)/(np.linalg.norm(v)**2)
    inv_row_norms = np.reshape(1/np.linalg.norm(true_matrix, axis=1), [h, 1])
    true_matrix = true_matrix * inv_row_norms # rescale row norms to 1
    Q = true_matrix+U*inv_snr/np.sqrt(w)
    return Q

def assess_Q(Q):
    true_residual = np.linalg.norm(Q @ v)
    _, _, V = np.linalg.svd(Q)
    best_sv = V[-1, :]/V[-1, 0]
    return true_residual, best_sv

#Q = generate_random_Q(sigma)
#true_residual, best_sv = assess_Q(Q)
#print("Q:", Q)
#print("Residual with true solution:", true_residual)
#print("Best singular vector:", best_sv)

In [29]:
#%%prun

sigma = 1/8 # (signal to noise ratio)^-1
N_trials = 100

verbose = False

np.random.seed(1)

pr_vec = []
sr_vec = []
tsr_vec = []
for i in range(N_trials):
    print("i: ", i)
    scaler = Scaler(sub_inds=None, char_sizes=np.ones([w]), row_norms=None, unit_rows=False)
    #init = Initializer(method='combinatorial', start_k=2)
    #init = Initializer(method='combinatorial', start_k=9999)
    init = Initializer(method='power', start_k=10)
    res = Residual(residual_type='matrix_relative')
    iterator = ModelIterator(max_k=20, backward_forward=True, max_passes=3, use_best_solution=True)
    #iterator = ModelIterator(max_k=100, backward_forward=False, max_passes=1)
    thres = Threshold(threshold_type=None, n_terms=10)

    Q = generate_random_Q(sigma)
    true_residual, best_sv = assess_Q(Q)
    reg_result = sparse_reg_bf(Q, scaler, init, res, iterator, thres, verbose=verbose)
    xi = reg_result.xi/np.linalg.norm(reg_result.xi)
    xi2, _ = regress(Q, list(range(k)), np.ones([w]))
    xi2 = xi2/np.linalg.norm(xi2)
    thresholded_inds = np.argpartition(np.abs(best_sv), -10)[-10:]
    
    achieved_residual = np.linalg.norm(Q @ xi)
    adj_true_residual = np.linalg.norm(Q @ xi2)
    #print("Reported lambda:", reg_result.lambd, "and xi:", reg_result.xi[:10])
    #print(achieved_residual, "vs", adj_true_residual)
    performance_ratio = achieved_residual/adj_true_residual
    support_recovery = sum(abs(reg_result.xi[0:k])>0)/k
    TSR = sum(thresholded_inds<10)
    
    pr_vec.append(performance_ratio)
    sr_vec.append(support_recovery)
    tsr_vec.append(TSR/10)
    #print("Recovered coefficient vector:", reg_result.xi)
    #print("Recovered support:", np.where(abs(reg_result.xi>0)))
    #print("Ratio of achieved residual to residual achieved by true solution: ", performance_ratio)
    #print("Fraction of correct support recovered: ", support_recovery)

print("Mean performance ratio:", np.round(np.mean(pr_vec), 4))
print("Mean support recovery:", np.round(np.mean(sr_vec), 3))
print("Mean support recovery by simple thresholding:", np.round(np.mean(tsr_vec), 3))

i:  0
i:  1
i:  2
i:  3
i:  4
i:  5
i:  6
i:  7
i:  8
i:  9
i:  10
i:  11
i:  12
i:  13
i:  14
i:  15
i:  16
i:  17
i:  18
i:  19
i:  20
i:  21
i:  22
i:  23
i:  24
i:  25
i:  26
i:  27
i:  28
i:  29
i:  30
i:  31
i:  32
i:  33
i:  34
i:  35
i:  36
i:  37
i:  38
i:  39
i:  40
i:  41
i:  42
i:  43
i:  44
i:  45
i:  46
i:  47
i:  48
i:  49
i:  50
i:  51
i:  52
i:  53
i:  54
i:  55
i:  56
i:  57
i:  58
i:  59
i:  60
i:  61
i:  62
i:  63
i:  64
i:  65
i:  66
i:  67
i:  68
i:  69
i:  70
i:  71
i:  72
i:  73
i:  74
i:  75
i:  76
i:  77
i:  78
i:  79
i:  80
i:  81
i:  82
i:  83
i:  84
i:  85
i:  86
i:  87
i:  88
i:  89
i:  90
i:  91
i:  92
i:  93
i:  94
i:  95
i:  96
i:  97
i:  98
i:  99
Mean performance ratio: 0.9998
Mean support recovery: 0.994
Mean support recovery by simple thresholding: 0.95


In [31]:
print("Max PR:", max(pr_vec))
print("Min PR:", min(pr_vec))
print("Max SR:", max(sr_vec))
print("Min SR:", min(sr_vec))
ops = [(pr, i) for i, pr in enumerate(pr_vec) if pr<1-1e-6]
cf = [(pr, sr, i) for i, pr, sr in zip(list(range(N_trials)), pr_vec, sr_vec) if sr<0.2]
print(f"{len(ops)} outperformances:", ops)
print(f"{len(cf)} catastrophic failures:", cf)


Max PR: 1.0000000000000002
Min PR: 0.9891447580223384
Max SR: 1.0
Min SR: 0.9
6 outperformances: [(np.float64(0.9951930185682167), 16), (np.float64(0.9971311129076699), 21), (np.float64(0.9986740923489786), 30), (np.float64(0.9992657312782306), 44), (np.float64(0.9891447580223384), 72), (np.float64(0.9997766030054969), 91)]
0 catastrophic failures: []


In [5]:
print(best_sv)
print(reg_result.xi)

[ 1.000e+00  9.587e-01  8.129e-01  7.440e-01  5.920e-01  5.380e-01
  4.035e-01  3.192e-01  2.652e-01  1.003e-01 -1.953e-02  1.600e-04
 -1.475e-02 -1.703e-02  3.509e-02  1.594e-04 -3.097e-03  3.957e-02
  5.509e-03  2.696e-02 -2.747e-02  7.459e-03  3.698e-02  1.422e-03
 -7.654e-02 -4.949e-02  2.637e-03  1.437e-02 -2.360e-02  4.498e-03
 -3.833e-02  8.387e-03 -2.913e-02 -1.978e-02 -2.387e-02  1.601e-02
 -3.186e-03 -1.353e-02  3.871e-02 -3.767e-02  6.803e-02 -2.466e-02
  5.050e-02 -1.016e-02 -5.159e-02  2.726e-02  1.769e-02  5.265e-02
  4.036e-02 -7.968e-03  1.346e-02 -1.989e-02  4.812e-02  7.353e-03
 -5.088e-03 -8.953e-02 -2.083e-02  1.893e-02 -4.518e-02  4.243e-03
 -7.277e-03 -4.708e-02 -3.453e-02  1.021e-02  2.347e-02 -7.557e-02
  6.840e-02  2.028e-02  1.902e-02  1.470e-03 -3.804e-02  3.499e-02
  9.100e-02 -3.086e-02 -3.382e-02 -1.701e-03  3.269e-02 -6.526e-02
 -1.271e-02 -1.179e-02 -1.482e-02  2.091e-02 -3.310e-03 -2.714e-02
 -1.341e-02 -1.000e-03 -2.282e-02  1.838e-02 -2.697e-02 -1.670

In [6]:
SNR = 3, v3

combinatorial k=9999, no bf:
Mean performance ratio: 0.999
Mean support recovery: 0.962

SyntaxError: invalid syntax (2166125581.py, line 3)

In [ ]:
SNR = 3, v2

combinatorial k=2, no bf (=1 bf):
Mean performance ratio: 1.299
Mean support recovery: 0.811

(k=3:
Mean performance ratio: 1.06
Mean support recovery: 0.905)

..., 2 bf:
Mean performance ratio: 1.194
Mean support recovery: 0.888

..., 3 bf:
Mean performance ratio: 1.202
Mean support recovery: 0.859

..., 4 bf:
Mean performance ratio: 1.183
Mean support recovery: 0.894

..., 5 bf:
Mean performance ratio: 1.202
Mean support recovery: 0.859

[...]

..., 10 bf:
Mean performance ratio: 1.183
Mean support recovery: 0.894

---

combinatorial k=9999, no bf:
Mean performance ratio: 0.999
Mean support recovery: 0.962

---

power (k=10), no bf:
Mean performance ratio: 1.068
Mean support recovery: 0.901

power (k=15), no bf:
Mean performance ratio: 1.014
Mean support recovery: 0.934

power (k=20), no bf:
Mean performance ratio: 1.005
Mean support recovery: 0.949

power (k=20), more bf: (no better)

power (k=30), no bf:
Mean performance ratio: 1.002
Mean support recovery: 0.957

power (k=40), no bf:
Mean performance ratio: 1.0
Mean support recovery: 0.961

In [ ]:
SNR = 3

start_k=9999, no bf: (245 sec)
Mean performance ratio: 1.001
Mean support recovery: 0.994

start_k=2, no bf: (22 sec)
Mean performance ratio: 1.305
Mean support recovery: 0.815

start_k=2, 2 bf passes:
Mean performance ratio: 1.266
Mean support recovery: 0.865

start_k=2, 10 bf passes: (65 sec)
Mean performance ratio: 1.158
Mean support recovery: 0.895

power (k=20), no bf: (3 sec)
Mean performance ratio: 2.078
Mean support recovery: 0.567

power (k=20), 10 bf passes: (55 sec)
Mean performance ratio: 1.076
Mean support recovery: 0.946

In [ ]:
SNR = 2 (most PRs compared to the true solution, not adjusted solution on true support!)

start_k=9999, no bf: (255 sec)
Mean performance ratio: 0.892 / 0.891 (actually 0.997)
Mean support recovery: 0.954 / 0.971

start_k=2, no bf:
Mean performance ratio: 1.072
Mean support recovery: 0.74

start_k=2, 2 bf passes:
Mean performance ratio: 1.065
Mean support recovery: 0.75 / 0.793

start_k=2, 10 bf passes: (65 sec)
Mean performance ratio: 1.009 / 0.983
Mean support recovery: 0.847 / 0.841
 
power (k=20), no bf:
Mean performance ratio: 1.684
Mean support recovery: 0.328

power (k=40), no bf:
Mean performance ratio: 1.651
Mean support recovery: 0.319

power (k=20), 2 bf passes:
Mean performance ratio: 1.118
Mean support recovery: 0.723

power (k=20), 10 bf passes: (56 sec)
Mean performance ratio: 0.944 / 0.977
Mean support recovery: 0.898 / 0.848

power (k=20), 50 bf passes: (263 sec)
Mean performance ratio: 0.975
Mean support recovery: 0.864

power (k=40), 10 bf passes: (226 sec)
Mean performance ratio: 0.963
Mean support recovery: 0.857